In [3]:
def open_file_and_read_data(term_id, filename, catalog):
    start_offset_position = 0
    len_position_after_offset = 1
    with open(filename, "r") as file:
        file.seek(catalog[term_id][start_offset_position])
        row_reader = file.read(catalog[term_id][len_position_after_offset])
    read = parser_returns_a_list(row_reader)
    return read


In [4]:
def temp_parsed_inverted_list(term_id, filename, catalog, i_or_j):
    term_dict_temp = {}
    read = open_file_and_read_data(term_id, filename, catalog)
    term_dict_temp[term_id] = read[term_id]
    i_or_j += 1
    return i_or_j, term_dict_temp
        

In [6]:
def merger1(catalog1, catalog2, filename1, filename2):
    key1 = list(catalog1.keys())
    key2 = list(catalog2.keys())
    key1.sort()
    key2.sort()
    new_catalog = {}
    recently_seek_added_address = False
    i, j = 0, 0
    new_filename = str(int(time.time())) + ".txt"
    while(i < len(key1) and j < len(key2)):
        temp_catalog = {}
        if key1[i] == key2[j]:
            term_dict_temp = {}    
            term_id = key1[i]
            read1 = open_file_and_read_data(term_id, filename1, catalog1)
            read2 = open_file_and_read_data(term_id, filename2, catalog2)
            read_merger = concatenate_two_term_inverted_lists(read1, read2, term_id)
            term_dict_temp[term_id] = read_merger
            i += 1
            j += 1
        elif key1[i] < key2[j]:
            term_id = key1[i]
            i, term_dict_temp = temp_parsed_inverted_list(term_id, filename1, catalog1, i)
        else:
            term_id = key2[j]
            j, term_dict_temp = temp_parsed_inverted_list(term_id, filename2, catalog2, j)
        temp_catalog = write_into_file(new_filename, term_dict_temp, recently_seek_added_address)
        new_catalog[term_id] = temp_catalog[term_id]
        recently_seek_added_address = sum(temp_catalog[term_id])
        
    while (i < len(key1)):
        temp_catalog = {}
        term_id = key1[i]
        i, term_dict_temp = temp_parsed_inverted_list(term_id, filename1, catalog1, i)
        temp_catalog = write_into_file(new_filename, term_dict_temp, recently_seek_added_address)
        new_catalog[term_id] = temp_catalog[term_id]
        recently_seek_added_address = sum(temp_catalog[term_id])
    
    while (j < len(key2)):
        temp_catalog = {}
        term_id = key2[j]
        j, term_dict_temp = temp_parsed_inverted_list(term_id, filename2, catalog2, j)
        temp_catalog = write_into_file(new_filename, term_dict_temp, recently_seek_added_address)
        new_catalog[term_id] = temp_catalog[term_id]
        recently_seek_added_address = sum(temp_catalog[term_id])
    return new_filename, new_catalog


In [ ]:
def merging(catalog1, catalog2, filename1, filename2):
    start_offset_position = 0
    len_position_after_offset = 1
    newn_catalog = {}
    recently_seek_added_address = False
    filename = str(int(time.time())) + ".txt"
    with open(filename1, "r") as file1:
        for term_id in catalog1:
            temp_catalog = {}
            term_dict_temp = {}
            file1.seek(catalog1[term_id][start_offset_position])
            row_reader = file1.read(catalog1[term_id][len_position_after_offset])
            read1 = parser_returns_a_list(row_reader)
            if term_id in catalog2.keys():
                with open(filename2, "r") as file2:
                    file2.seek(catalog2[term_id][start_offset_position])
                    row_reader = file2.read(catalog2[term_id][len_position_after_offset])
                    read2 = parser_returns_a_list(row_reader)
                    read_merger = concatenate_two_term_inverted_lists(read1, read2, term_id)
                    term_dict_temp[term_id] = read_merger
                    temp_catalog = write_into_file(filename, term_dict_temp, recently_seek_added_address)
                    catalog2.pop(term_id)
            else:
                term_dict_temp[term_id] = read1[term_id]
                temp_catalog = write_into_file(filename, term_dict_temp, recently_seek_added_address)
            newn_catalog[term_id] = temp_catalog[term_id]
            recently_seek_added_address = sum(temp_catalog[term_id])

    with open(filename2, "r") as file2:
        for term_id in catalog2.keys():
            term_dict_temp = {}
            file2.seek(catalog2[term_id][start_offset_position])
            row_reader = file2.read(catalog2[term_id][len_position_after_offset])
            read2 = parser_returns_a_list(row_reader)        
            term_dict_temp[term_id] = read2[term_id]    
            temp_catalog = write_into_file(filename, term_dict_temp, recently_seek_added_address)
            newn_catalog[term_id] = temp_catalog[term_id]    
            recently_seek_added_address = sum(temp_catalog[term_id])
        #del (catalog2)
    return filename, newn_catalog

In [ ]:

def concatenate_two_term_inverted_lists(read1, read2, term_id, shorter_version = True):
    read_two_entries = read1[term_id] + read2[term_id]
    if not shorter_version:
        read_two_entries.sort()
        i = 0
        flag = True
        while (i < len(read_two_entries) - 1):
            if read_two_entries[i][0] == read_two_entries[i + 1][0]:
                temp_list = [read_two_entries[i][0]] 
                if len(read_two_entries[i]) > len(read_two_entries[i + 1]):
                    count_pos_list = read_two_entries[i][1:]
                else:
                    count_pos_list = read_two_entries[i + 1][1:]
                read_two_entries[i] = [None]
                flag = False
                temp_list.extend(count_pos_list)
                read_two_entries[i + 1] = temp_list
                i += 2
            else:
                i += 1
        if flag == False:
            read_two_entries = [i for i in read_two_entries if i != [None]]    
    return read_two_entries